In [1]:
#imports 

import os 
import glob
from dotenv import load_dotenv
import gradio as gr

from langchain.document_loaders import DirectoryLoader , TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [2]:
MODEL = "gpt-4o-mini"
db_name = "vector_dbl"

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

# Preparing Documents

In [4]:
folders = glob.glob("Database/*")
def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding':'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder,glob="**/*.md",loader_cls = TextLoader, loader_kwargs= text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc,doc_type) for doc in folder_docs])
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks : {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Total number of chunks : 8
Document types found: {'store policy', 'misc info', 'working hours', 'store location', 'top staff', 'store info'}


# Embeddings

In [5]:
embeddings = OpenAIEmbeddings() 

if os.path.exists(db_name):  # checks for the existence of a directory or file named db_name
    Chroma(persist_directory=db_name, embedding_function = embeddings)
    # Chroma: A vector store implementation from langchain that uses Chroma to manage embeddings 
vectorstore = Chroma.from_documents(documents=chunks,embedding=embeddings,persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()}")

Vectorstore created with 88


In [6]:
# creating a vector store
vectorstore = Chroma.from_documents(documents= chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()}")

Vectorstore created with 96


In [7]:
collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1,include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 96 vectors with 1,536 dimensions in the vector store


In [8]:
model = ChatOpenAI(temperature=0.7)

memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)

#retrieving

retriever = vectorstore.as_retriever(search_kwargs={"k":25})



/var/folders/rx/ghbhygn97p30c1z12nfn7np80000gn/T/ipykernel_48867/2290987020.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)


# Rewrite-Retrieve-Read

this is a method we will use for query transformation Because the original query can not be always optimal to retrieve for the LLM, especially in the real world we first prompt an LLM to rewrite the queries, then conduct retrieval-augmented reading.

In [9]:
template = """
Provide a better search query for \
web search engine to answer the given question,
end \
the queries with '**'. Question: \
{x} Answer"
"""

prompt = ChatPromptTemplate.from_template(template)

rewrite_prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain import hub

rewrite_prompt = hub.pull("langchain-ai/rewrite")


In [11]:
# parser to remove the '**'

def _parse(text):
    return text.strip('"').strip("**")
    

In [12]:
rewriter = rewrite_prompt | ChatOpenAI(temperature=0.7) | StrOutputParser() | _parse


In [13]:
# we create a distracted query to test the function we created
distracted_query = "life is weird and meaningless , who is the general manager?"

In [14]:
qa_prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. 
    Use the following context to answer the question.
    Context: {context}
    Question: {x}
    Answer:"""
)

In [15]:
reader = qa_prompt | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()


rag_chain = {
    "x": RunnablePassthrough(),
    "context": rewriter | retriever
} | reader

response = rag_chain.invoke(distracted_query)
print(response)

The general manager is Sarah Thompson.


In [17]:
from __future__ import annotations
from typing import Iterable
import gradio as gr
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
import time

# === Custom Theme ===
class Seafoam(Base):
    def __init__(
        self,
        *,
        primary_hue: colors.Color | str = colors.violet,
        secondary_hue: colors.Color | str = colors.sky,
        neutral_hue: colors.Color | str = colors.green,
        spacing_size: sizes.Size | str = sizes.spacing_sm,
        radius_size: sizes.Size | str = sizes.radius_lg,
        text_size: sizes.Size | str = sizes.text_lg,
        font: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"),
            "ui-sans-serif",
            "sans-serif",
        ),
        font_mono: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("IBM Plex Mono"),
            "ui-monospace",
            "monospace",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )
        super().set(
            button_border_width="0px",
            checkbox_label_border_width="1px",
            button_transform_hover="scale(1.02)",
            button_transition="all 0.1s ease-in-out",
            slider_color="*primary_400",
            button_primary_background_fill="linear-gradient(120deg, *secondary_500 0%, *primary_300 60%, *primary_400 100%)",
            button_primary_background_fill_hover="linear-gradient(120deg, *secondary_400 0%, *primary_300 60%, *primary_300 100%)",
            button_primary_text_color="*button_secondary_text_color",
            button_secondary_background_fill="linear-gradient(120deg, *neutral_300 0%, *neutral_100 60%, *neutral_200 100%)",
            button_secondary_background_fill_hover="linear-gradient(120deg, *neutral_200 0%, *neutral_100 60%, *neutral_100 100%)",
            checkbox_label_background_fill_selected="linear-gradient(120deg, *primary_400 0%, *primary_300 60%, *primary_400 100%)",
            checkbox_label_border_color_selected="*primary_400",
            checkbox_background_color_selected="*primary_400",
            checkbox_label_text_color_selected="*button_secondary_text_color",
            slider_color_dark="*primary_500",
            button_primary_background_fill_dark="linear-gradient(120deg, *secondary_600 0%, *primary_500 60%, *primary_600 100%)",
            button_primary_background_fill_hover_dark="linear-gradient(120deg, *secondary_500 0%, *primary_500 60%, *primary_500 100%)",
            button_primary_text_color_dark="*button_secondary_text_color",
            button_secondary_background_fill_dark="linear-gradient(120deg, *neutral_700 0%, *neutral_600 60%, *neutral_700 100%)",
            button_secondary_background_fill_hover_dark="linear-gradient(120deg, *neutral_600 0%, *neutral_600 60%, *neutral_700 100%)",
            checkbox_label_background_fill_selected_dark="linear-gradient(120deg, *primary_600 0%, *primary_500 60%, *primary_600 100%)",
            checkbox_label_border_color_selected_dark="*primary_600",
            checkbox_background_color_selected_dark="*primary_600",
            checkbox_label_text_color_selected_dark="*button_secondary_text_color",
            block_shadow="*shadow_drop_lg",
            button_secondary_shadow_hover="*shadow_drop_lg",
            button_primary_shadow_hover="0 1px 3px 0 *primary_200, 0 1px 2px -1px *primary_200",
            button_secondary_shadow_dark="none",
            button_primary_shadow_dark="none",
        )

seafoam = Seafoam()

# === RAG Chat Function ===
def chat(question, history):
    try:
        result = rag_chain.invoke({"x": question})
        # if result is a string, return it directly
        if isinstance(result, str):
            return result
        # if result is dict, pull out the answer
        elif isinstance(result, dict):
            return result.get("answer", str(result))
        else:
            return str(result)
    except Exception as e:
        return f"⚠️ Error: {str(e)}"
# === Gradio Interface with Theme ===
with gr.Blocks(theme=seafoam) as demo:
    gr.Markdown("Aurora Home & Kitchen Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Ask me anything")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        answer = chat(message, chat_history)
        chat_history.append((message, answer))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(inbrowser=True)

/var/folders/rx/ghbhygn97p30c1z12nfn7np80000gn/T/ipykernel_48867/4073289722.py:95: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [17]:
%%ai

UsageError: Cell magic `%%ai` not found.
